Learning a new command
=========================
Here the agent reads one example from its own README file and do online research to implement a new command, then try and use it immediately

In [1]:
from lm_agent.agent import Agent
agent = Agent(model='gpt-4', work_dir='gpt-4', config={'command': {'reader': {'summarization_model': 'gpt-4'}}}, disable=('delegate'), essential_only=False)

### First show the example and ask it to code up the new command


In [2]:
print(agent.instruct("""Read the section 'How to implement a new command' from 
`https://raw.githubusercontent.com/virtualzx-nad/easy_llm_agents/main/README.md` to get the code and instructions for implementing new commands verbatim,
then write a command to get weather forecast for a date in the near future.
When you design your command make sure it does not return too much information to blow up the token limit.
If you choose to use an API, make sure you read the doc to understand the parameters and returns structure, and think about what need to be passed and returned for you command.
I cannot provide you with API keys due to privacy restrictions, but the code can directly retrieve the API keys from `self.metadata`. 
Do not use APIs that are not free, and tell me what API keys need to be passed into metadata if you need them.  
The command will be created or updated as soon as you submit and execute the python code, but make sure to notify me and get approval before you start using them."""))

<Permissive Overseer>think requested. "Plan out steps to fulfill user's request". GRANTED.
<Permissive Overseer>reader requested. "Read 'How to implement a new command' section verbatim". GRANTED.
<Command think> {'thoughts': ["Read the section 'How to implement a new command' from the provided URL verbatim", 'Use `reader` command to read the section from the URL', 'Design a command to get weather forecast for a date in the near future', 'Use `python` command to create or update the command', 'Notify user and get approval before using the command'], 'notes': []}
<Command reader> {'url': 'https://raw.githubusercontent.com/virtualzx-nad/easy_llm_agents/main/README.md', 'instruction': 'How to implement a new command'}
<Command reader> {'lines': 82, 'tokens': 1423, 'n': 1}
Timeout from OpenAI.  Likely service dis down.  Waiting 30s to retry. 
<Permissive Overseer>think requested. "Plan out steps to design a weather forecast command". GRANTED.
<Permissive Overseer>search requested. "Find a 

### Add API keys per its request
Add the requested API key to metadata, if the agent requested it.  
There are a few different APIs in this area and the agent usually choose from OpenWeatherMap, WeatherAPI and Open-Meteo APIs. 
If it chose one that is free but needs an API key, it will tell you to supply it here and you need to navigate to the site and get an 
API key then store it into metadata objects below. 
Do not tell the agent what the key is as that is probably not safe behavior.
If it chose one that doesn't need API key (say Open Mateo) you can comment this block out and just proceed to later ones.

In [3]:
# agent.metadata['openweathermap_api_key'] =

### Check if new command is there
Check the command list to make sure the new command is online

In [3]:
print(agent.generate_command_list())

Your responses must be a command in the form of a Python dict with fields:
    - notes:  Use this field to reason and plan. Notes are invisible to others.
    - command:  name of command
    - summary:  purpose of command in one sentence
    - content:  content that is passed to the worker.
If a command did not work, note why it failed and try a different approach instead of keep repeating a failed one.Respond with command only and do not give any explanations. - `answer`: Response to the user. The actual text given to user should be in the content field as a string. If one approach requires information from the user, try an alternatively approach instead of asking the user for information if possible.
 - `search`: Search Goole to get top results and URLs to expand your knowledge. You must provide the exact search phrase and nothing else in `query` , and optionally tbs and tbm only when warranted. 
 - `reader`: Obtain info from a page based on URL or filename. Put the full url in `url`

### Now ask it to run the new command
Now it is in metadata we can tell it to continue.  Then ask the question and see it uses the command

In [4]:
print(agent.instruct("""OK I have added the API key to the metadata.  
Now you can invoke command `weather_forecast` to check if it works correctly. 
What's the weather in San Francisco the day after tomorrow?"""))

APIConnectionError from OpenAI.  Connection was reset.  Waiting 10s to retry. 
<Permissive Overseer>think requested. "Plan out steps to get the weather forecast for San Francisco". GRANTED.
<Permissive Overseer>search requested. "Find the latitude and longitude of San Francisco". GRANTED.
<Command think> {'thoughts': ['Find the latitude and longitude of San Francisco', 'Use `search` command to find the latitude and longitude of San Francisco', 'Calculate the date for the day after tomorrow', 'Use `python` command to calculate the date for the day after tomorrow', 'Invoke `weather_forecast` command with the latitude, longitude, and date'], 'notes': []}
<Command search> {'query': 'San Francisco latitude and longitude', 'size': 3, 'tbs': None, 'tbm': None}
<Command search> {'num_results': 3, 'result_len': 2261, 'titles': ['Where is San Francisco, CA, USA on Map Lat Long ...', 'Description', 'San Francisco, California - OpenStreetMap Wiki']}
Return is too long. Will summarize
<Permissive O

### Under the hood

Now we print out the full conversation history to see what went on under the hood

In [5]:
for entry in agent.conversation.raw_history:
    print(f"{entry['role']}: {entry['content']}")

user: Use commands to fulfill instruction: `Read the section 'How to implement a new command' from 
`https://raw.githubusercontent.com/virtualzx-nad/easy_llm_agents/main/README.md` to get the code and instructions for implementing new commands verbatim,
then write a command to get weather forecast for a date in the near future.
When you design your command make sure it does not return too much information to blow up the token limit.
If you choose to use an API, make sure you read the doc to understand the parameters and returns structure, and think about what need to be passed and returned for you command.
I cannot provide you with API keys due to privacy restrictions, but the code can directly retrieve the API keys from `self.metadata`. 
Do not use APIs that are not free, and tell me what API keys need to be passed into metadata if you need them.  
The command will be created or updated as soon as you submit and execute the python code, but make sure to notify me and get approval befo

### Check how much this has cost. 

Not cheap for a weather forecast, but decent for coding a new functionality

In [7]:
print(f'Total OpenAI cost: ${agent.model.total_cost():.2f}')

Total OpenAI cost: $1.68
